In [1]:
from collections import namedtuple
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallBackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from deepClassifier.constant import *
from deepClassifier.utils import read_yaml,create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_call_back_config(self) -> PrepareCallBackConfig:
        config = self.config.prepare_callbacks
        params = self.params
        
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir),Path(model_ckpt_dir)])

        prepare_call_back_config = PrepareCallBackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)

        )

        return prepare_call_back_config

In [7]:
from urllib import request
import os
from zipfile import ZipFile
import tensorflow as tf
import time


class PrepareCallBack:
    def __init__(self,config:PrepareCallBackConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
       
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_model_filepath,
            save_best_only= True
            )

        
    def get_call_back(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [8]:
try:
    config = ConfigurationManager()
    prepare_call_back_config = config.get_prepare_call_back_config()
    prepare_call_back = PrepareCallBack(config=prepare_call_back_config)
    callback_list = prepare_call_back.get_call_back()
    

except Exception as e:
    raise e

[2022-10-12 15:50:37,080: INFO: common: yaml file: configs\config.yaml loaded successfully
[2022-10-12 15:50:37,085: INFO: common: yaml file: params.yaml loaded successfully
[2022-10-12 15:50:37,088: INFO: common: created directory at: artifacts
[2022-10-12 15:50:37,091: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
[2022-10-12 15:50:37,093: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir
